In [3]:
# Instalar o langchain e o langchain_openai
! pip install langchain langchain_openai

# Instalar pandas
! pip install pandas

# json é um módulo padrão do Python, então não precisa ser instalado separadamente


In [8]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI
import pandas as pd
import json

# Ler a variável de ambiente
api_key = os.getenv('OPENAI_API_KEY')

In [10]:


# Configurar o cliente OpenAI corretamente
client = ChatOpenAI(
    api_key=api_key,
    model="gpt-3.5-turbo",
    temperature=0.7
)

# Definir o esquema do JSON esperado como saída
response_schemas = [
    ResponseSchema(name="product_summary", description="Um nome simplificado e resumido do produto."),
    ResponseSchema(name="category", description="A categoria do produto em uma palavra."),
    ResponseSchema(name="material", description="O material predominante do produto em uma palavra."),
    ResponseSchema(name="functions", description="As funções principais do produto, escritas de forma consistente para produtos similares em uma palavra."),
    ResponseSchema(name="compatibility", description="A compatibilidade do produto com outros itens, se aplicável."),
    ResponseSchema(name="price", description="Um valor estimado numérico no formato inteiro."),
    ResponseSchema(name="other_details", description="Outros detalhes relevantes do produto.")
]

# Criar um parser com LangChain para validar o formato de saída
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Função para extrair características (features) do produto com LLM e validar com o LangChain
def extrair_features_com_parser(titulo, descricao):
    prompt_template = """
    Titulo: {titulo}
    Descrição do Produto: {descricao}
    
    Quais são as principais features do produto? Resuma as informações abaixo em um formato JSON com as seguintes diretrizes:
    - Simplifique o nome do produto.
    - Cada feature deve ser descrita com uma única palavra.
    - Descreva cada categoria com uma única palavra. Exemplo "category" : "mobile"
    - Produtos similares devem ter descrições consistentes de funções (exemplo: use termos similares para slots, suportes, etc.).
    - Para o material, se houver mais de um, selecione o material que mais representa o produto.
    - Estime um valor para o preço do produto em dólares americanos.
    - As funções principais do produto devem ser brevemente descritas com uma palavra apenas.
    - Inclua outros detalhes relevantes sobre o produto com uma palavra.
    - Cada valor no esquema abaixo deve ter apenas uma palavra.

    O formato do JSON deve ser o seguinte:
    {{
        "product_summary": "...",
        "category": "...",
        "material": "...",
        "functions": {{
            "main_function": "...",
            "additional_function_1": "...",
            "additional_function_2": "..."
        }},
        "compatibility": "...",
        "price": int,
        "other_details": "..."
    }}
    """

    # Substituir as variáveis no prompt
    prompt = prompt_template.format(titulo=titulo, descricao=descricao)

    # Chamar o LLM usando a função invoke corretamente, passando o parâmetro 'input'
    response = client.invoke(
        input=[
            {"role": "system", "content": "Você é um assistente que extrai características de descrições de produtos."},
            {"role": "user", "content": prompt}
        ]
    )

    # O conteúdo da resposta pode ser acessado diretamente como `response.content`
    try:
        parsed_response = output_parser.parse(response.content.strip())
        return parsed_response
    except Exception as e:
        print(f"Erro ao validar o formato JSON: {e}")
        return None

# Carregar o CSV "linhas_validas.csv"
df = pd.read_csv("dataset/linhas_validas.csv")

# Aplicar a extração de características com o parser para garantir o formato correto
df['features'] = df.apply(lambda row: extrair_features_com_parser(row['title'], row['text']), axis=1)

# Manter todas as features como um dicionário em uma única coluna
df['features'] = df['features'].apply(lambda x: json.dumps(x) if x is not None else '{}')

# Salvar o DataFrame atualizado em um novo arquivo JSON
output_json = "dataset/linhas_validas_com_features.json"
df.to_json(output_json, orient='records', lines=True)

print(f"Arquivo JSON '{output_json}' salvo com sucesso com as features em uma única coluna no formato dicionário.")


Arquivo JSON 'dataset/linhas_validas_com_features.json' salvo com sucesso com as features em uma única coluna no formato dicionário.
